In [ ]:
from google.colab import drive
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re

from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Concatenate, Reshape
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import Model, Input
from tensorflow.keras.optimizers import SGD, Nadam
from tensorflow.keras.losses import MeanAbsoluteError


In [ ]:
# Broken dataset link: https://drive.google.com/drive/folders/1IRhybT-PCuwD3pIvzRMxuEByA7p5Iz4K?usp=sharing
# Train & test dataset link: https://drive.google.com/drive/folders/1zWYJvkjkiBbfQuTBSTp3Btk-m8uGiSwN?usp=sharing
drive.mount('/content/gdrive')

In [ ]:
x = glob.glob("/content/gdrive/MyDrive/*/")

In [ ]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 30%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [ ]:
def read_and_resize(filename: str, grayscale: bool = False, fx: float = 1.0, fy: float = 1.0):
    if grayscale:
        img_result = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
        imgbgr = cv2.imread(filename, cv2.IMREAD_COLOR)
        # convert to rgb
        img_result = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2RGB)
    # resize
    if fx != 1.0 and fy != 1.0:
        img_result = cv2.resize(img_result, None, fx=fx, fy=fy, interpolation=cv2.INTER_CUBIC)
    return img_result

In [ ]:
def show_in_row(list_of_images: list, titles: list = None, disable_ticks: bool = False):
    count = len(list_of_images)
    for idx in range(count):
        subplot = plt.subplot(1, count, idx + 1)
        if titles is not None:
            subplot.set_title(titles[idx])

        img = list_of_images[idx]
        cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
        subplot.imshow(img, cmap=cmap)
        if disable_ticks:
            plt.xticks([]), plt.yticks([])
    plt.show()

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text

In [ ]:
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
def make_model():
    # coarse branch
    in_coarse = Input(shape=(240, 320, 3))
    c1 = Conv2D(filters=96, kernel_size=15, strides=5, activation='tanh')(in_coarse)
    c2 = MaxPool2D(pool_size=2, strides=1)(c1)
    c3 = Conv2D(filters=128, kernel_size=5, strides=3, activation='tanh')(c2)
    c4 = Conv2D(filters=128, kernel_size=3, strides=1, activation='tanh')(c3)
    c5 = Conv2D(filters=64, kernel_size=3, strides=1, activation='tanh')(c4)
    c6 = Flatten()(c5)
    c7 = Dense(5120, activation='tanh')(c6)
    c8 = Dense(4800)(c7)
    c9 = Reshape((60, 80, 1), input_shape=(4800,), name="coarse_out")(c8)

    # fine branch
    in_shape_fine = Input(shape=(240, 320, 3))
    f1 = Conv2D(filters=59, kernel_size=5, strides=2, padding="same", activation='tanh')(in_shape_fine)
    f2 = MaxPool2D(pool_size=2, name="fine_pool")(f1)
    f3 = Concatenate(axis=-1)([f2, c9])
    f4 = Conv2D(filters=60, kernel_size=5, strides=2,  padding="same", activation='tanh')(f3)
    f5 = Conv2D(filters=1, kernel_size=3, padding="same", name="Fine_out")(f4)

    model = Model(inputs=[in_coarse, in_shape_fine], outputs=f5, name="CF_model")
    return model

In [ ]:
dirs = glob.glob("/content/gdrive/MyDrive/DATASET/Train/Images/*/")

# model = make_model()
model = load_model('/content/gdrive/MyDrive/Model/CF_model.h5')
dirs.sort(key=natural_keys)
'''
broken folders:
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/basements/basement_0001c/
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/bathrooms_part1/bathroom_0002/
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/dining_rooms_part1/dining_room_0002/
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/dining_rooms_part1/dining_room_0006/
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/dining_rooms_part1/dining_room_0010/
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/dining_rooms_part1/dining_room_0011/
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/dining_rooms_part1/dining_room_0013/
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/dining_rooms_part1/dining_room_0017/
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/libraries/library_0004/
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/study_rooms/study_room_0001/
/content/gdrive/MyDrive/DATASET_BROKEN/Train/Images/study_rooms/study_room_0003/
'''

dir_start_index = 0
subdir_start_index = 0
'''
Dataset folders:
  basements
  bathrooms
  cafe
  dining rooms
  libraries
  study rooms
'''
for d in range(dir_start_index, len(dirs)):
    subdirs = glob.glob(dirs[d]+"*/")
    subdirs.sort(key=natural_keys)

    subdirs_len = len(subdirs)
    subdirs_counter = 1
    for s in range(subdir_start_index, len(subdirs)):
        print(str(subdirs_counter) +"/"+str(subdirs_len) + "\n" + subdirs[s] + "\nReading...")
        xfiles = glob.glob(subdirs[s]+"*.png")
        xfiles.sort(key=natural_keys)

        # counters of pairwise depth and color images
        bad_count = 0
        ok_count = 0
        color_images = []
        depth_images = []
        xfiles_len = len(xfiles)
        xfiles_counter = 0

        out = display(progress(0, 100), display_id=True)
        for f in xfiles:
            # depth images path assemble
            tmp = f.split("/")
            tmp[6] = "Depth"
            d_f = '/'.join(tmp)

            # not all color images have depth accordance in original dataset
            if not os.path.exists(d_f):
                bad_count = bad_count + 1
            else:
                ok_count = ok_count +1

                # color image 1/2 of original size
                color_images.append(read_and_resize(f, False, fx=0.5, fy=0.5))
                # depth image 1/8 of original size
                depth_images.append(read_and_resize(d_f, True, fx=0.0625, fy=0.0625))

            xfiles_counter += 1
            percent = xfiles_counter / xfiles_len * 100
            out.update(progress(percent, 100))

        subdirs_counter += 1

        # Read statistics of bad and good pairs
        print("Data reading result:")
        print(str(bad_count)+" images without depth pair")
        print(str(ok_count)+" full pairs")

        color_images_n = np.array(np.float32(color_images) / 255)
        depth_images_n = np.array(depth_images)
        # X_data - train feature vector
        # Y_data - train labels image
        print("X_data:\n"+str(color_images_n.shape))
        print("Y_data:\n"+str(depth_images_n.shape))

        
        # Data augmentation
        generator = ImageDataGenerator(rotation_range=5,
                                       zoom_range=[1.0, 1.5],
                                       horizontal_flip=True,
                                       validation_split=0.2,
                                       )
        generator.fit(color_images_n)
        
        # model build
        model.compile(optimizer=Nadam(1e-2),
                      loss=MeanAbsoluteError(),
                      metrics=['accuracy'])
        
        # model training
        model.fit(generator.flow([color_images_n, color_images_n],
                                 depth_images_n,
                                 batch_size=128),
                  validation_data=generator.flow([color_images_n, color_images_n],
                                                 depth_images_n,
                                                 batch_size=128,
                                                 subset='validation'),
                  steps_per_epoch=len(color_images_n) / 128,
                  epochs=300,
                  verbose=2,
                  batch_size=128, )
        
        # model is saved after each subdirectories iteration
        print("Saving model...")
        model.save('/content/gdrive/MyDrive/Model/CF_model.h5')

In [ ]:
model = load_model('/content/gdrive/MyDrive/Model/CF_model.h5')

s = "/content/gdrive/MyDrive/DATASET/Test/Images/study_rooms/study_room_0007/"

print(s + "\nReading...")
xfiles = glob.glob(s + "*.png")
xfiles.sort(key=natural_keys)

# counters of pairwise depth and color images
count = 0
ok_count = 0
color_images = []
depth_images = []
xfiles_len = len(xfiles)
xfiles_counter = 0

for f in xfiles:
    # depth images path assemble
    tmp = f.split("/")
    tmp[6] = "Depth"
    d_f = '/'.join(tmp)

    # not all color images have depth accordance in original dataset
    if not os.path.exists(d_f):
        count = count + 1
    else:
        ok_count = ok_count +1

        # color image 1/2 of original size
        color_images.append(read_and_resize(f, False, fx=0.5, fy=0.5))

        # color image 1/8 of original size
        depth_images.append(read_and_resize(d_f, True, fx=0.0625, fy=0.0625))
    xfiles_counter += 1
    percent = xfiles_counter / xfiles_len * 100
    out = display(progress(0, 100), display_id=True) 
    out.update(progress(percent, 100))

# Read statistics of bad and good pairs
print("Data reading result:")
print(str(count)+" images without depth pair")
print(str(ok_count)+" full pairs")

color_images_n = np.array(color_images)
depth_images_n = np.array(depth_images)


# X_data - test feature vector
# Y_data - test labels image
print("X_data:\n"+str(color_images_n.shape))
print("Y_data:\n"+str(depth_images_n.shape))

print("Evaluate on test data")
results = model.evaluate([color_images_n, color_images_n], depth_images_n, batch_size=128)
print("test loss, test acc:", results)

predictions = model.predict([color_images_n, color_images_n])
print(predictions.shape)